# Setup: Parameters, Imports

In [1]:
# Parameters

## Years
start_year = 2019
end_year = 2022
base_year = start_year
comparison_year = end_year
years = range(start_year, end_year + 1)


## Grouping
young_age_cutoff=25
old_age_threshold=65
income_groups = 'quintiles'
ses_groups = 'tertiles',

## Output
top_n = 5
comparison_level = 'primary'

## Folder Names
cex_data_folder="/Users/roykisluk/Downloads/Consumer_Expenditure_Survey/"
folder_names_pathname='Data_clean/CEX_folder_names.csv'
age_groups_pathname='Data_clean/age_groups.csv'
    
## Libraries
import pandas as pd
import pyreadstat  
import numpy as np
import matplotlib.pyplot as plt

# Import Data

In [2]:
# Load folder names
folder_names_df = pd.read_csv(folder_names_pathname)

# Load age groups
age_groups_df = pd.read_csv(age_groups_pathname)
young_age_group_id = age_groups_df[(age_groups_df['min_age'] <= young_age_cutoff) & (age_groups_df['max_age'] >= young_age_cutoff)].index[0] + 1
old_age_group_id = age_groups_df[(age_groups_df['min_age'] <= old_age_threshold) & (age_groups_df['max_age'] >= old_age_threshold)].index[0] + 1

# Load household data for each year
dfs_mb = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_HH_pathname = f"{cex_data_folder}{subfolder}/{subfolder}datamb.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_HH_pathname)
    df.columns = df.columns.str.lower()
    if 'gil' in df.columns:
        df.rename(columns={'gil': 'age_group'}, inplace=True)
    dfs_mb[year] = df

# Load individual data for each year
dfs_prat = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_IND_pathname = f"{cex_data_folder}{subfolder}/{subfolder}dataprat.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_IND_pathname)
    df.columns = df.columns.str.lower()
    if 'gil' in df.columns:
        df.rename(columns={'gil': 'age_group'}, inplace=True)
    dfs_prat[year] = df


FileNotFoundError: [Errno 2] No such file or directory: 'Data_clean/CEX_folder_names.csv'

# Grouping

In [ ]:
Groups = {}
for year in years:
    Groups[year] = pd.DataFrame(dfs_mb[year]['misparmb'].unique(), columns=['misparmb'])

In [ ]:
for year in years:
    dfs_mb_year = dfs_mb[year]
    dfs_prat_year = dfs_prat[year]

    nationality_map = {1: 'Jewish', 2: 'Arab'}
    observance_map = {1: 'Secular', 2: 'Conservative', 3: 'Religious', 4: 'Ultra-Orthodox', 5: 'Mixed'}

    Groups[year]['Nationality'] = dfs_mb_year['nationality'].map(nationality_map).fillna('Other')
    Groups[year]['Observance'] = dfs_mb_year['ramatdatiyut'].map(observance_map).fillna('Other')

    age_group_map = {age_group_id: 'Young' if age_group_id <= young_age_group_id else 'Old' if age_group_id >= old_age_group_id else 'Middle' for age_group_id in dfs_prat_year['age_group'].unique()}
    Groups[year]['Age_Group'] = dfs_prat_year.loc[dfs_prat_year['y_kalkali'] == 1, 'age_group'].map(age_group_map).values

    Groups[year]['Income_Decile'] = dfs_mb_year['decile'].fillna(0).astype(int)

    Groups[year]['Income_Quintile'] = pd.cut(dfs_mb_year['decile'], bins=[0, 2, 4, 6, 8, 10], labels=[1, 2, 3, 4, 5])

    Groups[year]['SES_Quintile'] = dfs_mb_year['cluster'].apply(lambda x: x if x in range(1, 6) else np.nan).fillna(0).astype(int)
    Groups[year]['SES_Tertile'] = dfs_mb_year['cluster'].apply(lambda x: 1 if x in [1, 2] else 2 if x == 3 else 3 if x in [4, 5] else np.nan).fillna(0).astype(int)

    Groups[year]['Children'] = dfs_mb_year['nefashotad18'].fillna(0).astype(int)
    Groups[year]['Family_Size'] = Groups[year]['Children'].apply(lambda x: 'no children' if x == 0 else '1 to 3' if x in [1, 2, 3] else '4 plus')


In [ ]:
Groups